In [ ]:
from ophyd import Signal, Device, EpicsSignal, EpicsSignalRO, Component as Cpt

from bluesky import RunEngine # executes plans like move, count, scan, .., handles logging, data collection etc

from bluesky.callbacks import LiveTable, LivePlot
from bluesky.plans import count, scan
from bluesky.plan_stubs import monitor, unmonitor, open_run, close_run, sleep

import time

In [ ]:
class Gaussmeter(Device):
   # READ
   bfield   = Cpt(EpicsSignalRO, 'rdBfield')
   volt     = Cpt(EpicsSignalRO, 'rdVolt')
   idn      = Cpt(EpicsSignalRO, 'rdIdn',       kind = "config")
   status   = Cpt(EpicsSignalRO, 'rdStatus',    kind = "config")

   # SWITCH
   onctrl   = Cpt(EpicsSignal,   'wrOnctrl',    kind = "config")
   mctrl =  Cpt(EpicsSignal,   'wrMctrl',    kind = "config")

   # COMMANDS 
   def switch(self, bi):
      """
      1: On
      0: Off
      """
      return self.onctrl.put(bi)

   def read_mag_field(self):
      """
      Read magn field
      """
      return self.bfield.get()
   
   def all(self):
      """
      Returns a directory
      """
      return {
         "mag_field": self.bfield.get(),
         "voltage" : self.volt.get(),
         "onctrl" : self.onctrl.get(),
         "idn" : self.idn.get(),
         "status" : self.status.get(),
         "mctrl" : self.status.get()
      }

rasp = Gaussmeter("G3:GM:", name = "rasp")
rasp.wait_for_connection()

# Switch the GM on
rasp.switch(1)
print(rasp.all())

{'mag_field': -119.375, 'voltage': 0.1674, 'onctrl': 1, 'idn': 'RASPY 4B HALL SENSOR (ADC1115 + Hall Se', 'status': 1, 'mctrl': 1}


In [103]:
def read_magnetic_field(filename = "data.txt"):
    list_of_b = []
    global exit
    with open(filename, "w") as f:
        f.write("B Field Data, Field in Gauss\n \n")
        f.write(str(rasp.all()) + "\n")
        f.write("\n ")

        while rasp.all()["mctrl"] == 1:
            b_val = rasp.read_mag_field()
            list_of_b.append(b_val)
            f.write(str(b_val) + "; ")
            f.flush()
            time.sleep(1)
        else:
            print("stop")
            
    return list_of_b
data = read_magnetic_field()

KeyboardInterrupt: 

In [ ]:
from bluesky import RunEngine
from bluesky.plan_stubs import monitor, unmonitor, open_run, close_run, sleep, mv

from databroker import Broker
import time

# Set up RunEngine
RE = RunEngine({})

# Attach databroker
db = Broker.named("temp")   # or your actual catalog
RE.subscribe(db.insert)

# Example plan using your Device (rasp)
def mag_field_plan(delay=1, num=10):
    yield from open_run(md)
    for i in range(num):
        yield from monitor()  # rasp is your Gaussmeter Device
        yield from sleep(delay)
    yield from close_run()

# Run the plan
uid = RE(mag_field_plan(delay=1, num=10))

# Retrieve data from databroker
hdr = db[uid]         # header object

# db[uid] returns a list of Run objects in v2
run = db[uid][0]            # take the first run
df = run.primary.read().to_dataframe()  # v2 method to get pandas DataFrame

print(df)
df.to_csv("mag_field_data.csv")


AttributeError: 'Header' object has no attribute 'primary'

CA.Client.Exception...............................................
    Context: "iocsc3ess3x.ews.bessy.de:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Wed Aug 27 2025 17:10:12.450405998
..................................................................
CA.Client.Exception...............................................
    Context: "iocsc3ess3x.ews.bessy.de:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Wed Aug 27 2025 17:10:49.991886801
..................................................................
